# utils

> Helper functions for estout package

In [ ]:
#|default_exp utils

Additional info  

In [ ]:
#|hide 
from nbdev.showdoc import *
from nbdev.test import *

In [ ]:
#|export
import functools
from typing import Dict, List, Tuple 
import pandas as pd 

In [ ]:
#| export
def rgetattr(obj, attr, *args):
    """Recursive getattr (for nested attributes)."""
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))

In [ ]:
#| export
def rsetattr(obj, attr, val):
    """Recursive setattr (for nested attributes)."""
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

In [ ]:
#| export
def default_formats() -> dict:
    """Default output formats for some very common statistics."""
    return  {'params':'{:.2f}', 
            'tstats':'{:.2f}', 
            'pvalues': '{:.3f}',
            'se':'{:.2f}', 
            'r2':'{:.3f}',
            'nobs':'{:.0f}'
            }

In [ ]:
default_formats()

{'params': '{:.2f}',
 'tstats': '{:.2f}',
 'pvalues': '{:.3f}',
 'se': '{:.2f}',
 'r2': '{:.3f}',
 'nobs': '{:.0f}'}

In [ ]:
#| export 
def get_stars(pvalues: pd.Series, # this is compared to key of 'stars' parameter to determine how many stars should be added
            stars: dict = {.1:'*',.05:'**',.01:'***'} # todo: default values to the left are star symbols that are not rendered correctly in markdown
            ) -> pd.Series:
    """For each pvalue, check the lowest key in 'stars' for which the pvalue is smaller than that key, and return the corresponding nr of stars."""

    param_names = list(pvalues.index)

    #Sort 'stars' by key (in reverse order)
    ks = list(stars.keys())
    ks.sort(reverse=True)
    stars = {k: stars[k] for k in ks}

    out = pd.Series('', index=param_names)
    for param in param_names:    
        for alpha in stars:
            if pvalues[param] < alpha:
                out[param] = stars[alpha]
                
    return out

In [ ]:
get_stars(pvalues= pd.Series([0.5, .03, 0.002], index=['x1','x2','x3']))

x1       
x2     **
x3    ***
dtype: object

In [ ]:
get_stars(pvalues= pd.Series([0.5, .03, 0.002], index=['x1','x2','x3']), 
          stars={0.4: '**', 0.9: '***'})

x1    ***
x2     **
x3     **
dtype: object

In [ ]:
#| export
def model_groups(column_group_names: Dict[str, List[int]], # keys are group titles, values are lists of column indices included in each group
                add_clines: bool=True # if True, adds lines below group names
                ) -> str:
    """Returns LaTex code needed to add at the top of the table in order to give names to groups of columns in the table."""
    
    group_names = ''
    group_lines = ''
    for key,value in column_group_names.items():
        if type(key) != str:
            raise TypeError('Each key in column_group_names must be a string')
        if type(value) != list:
            raise TypeError("Each value in column_group_names dict must be a list")
        if len(value) != 2:
            raise TypeError("Each value in column_group_names dict must contain two integers")
        
        value = sorted(value)
        group_names += '& \multicolumn{%i}{c}{%s} ' %(value[1]-value[0]+1,key)
        if add_clines:
            group_lines += '\cline{%s-%s} ' %(str(value[0]+1),str(value[1]+1))

    return group_names + ' \\\\ \n' + group_lines + ' \n'

In [ ]:
print(model_groups(column_group_names={'Group 1':[0,1], 'Group 2':[2,3]}))

& \multicolumn{2}{c}{Group 1} & \multicolumn{2}{c}{Group 2}  \\ 
\cline{1-2} \cline{3-4}  



In [ ]:
#| export
def tex_table_env(nr_columns: int, # number of columns in the table
                    env: str='tabularx' # latex tabular environment specification. either 'tabularx' or 'tabular*'
                    ) -> Tuple[str,str]:
    """Creates LaTex code to add at the top of the table to create the correct tabular environment."""

    if env=='tabularx':
        header = '\\begin{tabularx}{\\textwidth}{@{}l *{%i}{>{\centering\\arraybackslash}X}@{}}' %nr_columns 
        footer = '\\end{tabularx}'
    elif env=='tabular*':
        header = '\\begin{tabular*}{\\textwidth}{@{\extracolsep{\\fill}}l*{%i}{c}}' %nr_columns
        footer = '\\end{tabular*}'
    else:
        raise NotImplemented(f"LaTex tabular environment {env} has not yet been implemented in tex_table_env()")
    return header,footer 

In [ ]:
header,footer = tex_table_env(nr_columns=4, env='tabularx')
print(header)
print(footer)

\begin{tabularx}{\textwidth}{@{}l *{4}{>{\centering\arraybackslash}X}@{}}
\end{tabularx}


In [ ]:
header,footer = tex_table_env(nr_columns=4, env='tabular*')
print(header)
print(footer)

\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}l*{4}{c}}
\end{tabular*}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()